In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
import seaborn as sns
from datetime import datetime as dt  
from matplotlib.gridspec import GridSpec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score,auc
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv(r"C:\Users\Akash\Desktop\Project\airline-delay-and-cancellation-data-2009-2018\2009.csv")
print(df.shape)
df.sample(2)

(6429338, 28)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
5543446,2009-11-09,WN,884,SJC,SAN,1020,1020.0,0.0,8.0,1028.0,...,80.0,77.0,65.0,418.0,NaN,NaN,NaN,NaN,NaN,NaN
1173048,2009-03-09,OO,2853,EWR,MKE,1730,2035.0,185.0,21.0,2056.0,...,165.0,155.0,126.0,725.0,0.0,0.0,0.0,0.0,175.0,NaN


In [12]:
missing_percentage=df.isnull().sum()*100/len(df)
missing_value=pd.DataFrame({'missing_value': df.isnull().sum(),'missing_per': missing_percentage})

In [14]:
missing_value

,missing_value,missing_per
FL_DATE,0,0.000000
OP_CARRIER,0,0.000000
OP_CARRIER_FL_NUM,0,0.000000
ORIGIN,0,0.000000
DEST,0,0.000000
CRS_DEP_TIME,0,0.000000
DEP_TIME,82867,1.288889
DEP_DELAY,82867,1.288889
TAXI_OUT,85787,1.334305
WHEELS_OFF,85787,1.334305


In [3]:
df['YEAR'] = pd.DatetimeIndex(df['FL_DATE']).year
df['MONTH'] = pd.DatetimeIndex(df['FL_DATE']).month
df['DAY'] = pd.DatetimeIndex(df['FL_DATE']).day

In [4]:
t2=dt.now()
df['FLIGHT_DELAY'] = df['ARR_DELAY'].map( lambda x : 'YES' if x>0 else 'NO')
print(df.shape)
print(dt.now() -t2)

(6429338, 32)
0:00:03.222205


In [5]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'missing_count' : df.isnull().sum(),
                                 'percent_missing': percent_missing})

In [6]:
sample_per_class=df['FLIGHT_DELAY'].groupby(df['FLIGHT_DELAY']).count()
sample_per_class

FLIGHT_DELAY
NO     4026348
YES    2402990
Name: FLIGHT_DELAY, dtype: int64

In [5]:
delay_flights=df[df['FLIGHT_DELAY']=='YES']
print("Total flights: {}". format(df.shape[0]))
print("Total delay flights: {}". format(delay_flights.shape[0]))
print("Percentage of flights delay {}".format((delay_flights.shape[0]/df.shape[0])*100))

Total flights: 6429338
Total delay flights: 2402990
Percentage of flights delay 37.37538763710976


In [6]:
variables_to_remove=['OP_CARRIER_FL_NUM','CANCELLATION_CODE', 'NAS_DELAY',  'SECURITY_DELAY', 'CARRIER_DELAY', 'LATE_AIRCRAFT_DELAY', 
              'WEATHER_DELAY']
df.drop(variables_to_remove, axis=1, inplace=True)
print(df.columns)

Index(['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'DIVERTED',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'Unnamed: 27', 'YEAR', 'MONTH', 'DAY', 'FLIGHT_DELAY'],
      dtype='object')


In [7]:
df.drop('Unnamed: 27', axis=1, inplace=True)

In [8]:
df=df[df['MONTH']==1]
df=df[df['MONTH']==1]

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

clf = RandomForestClassifier(n_estimators=10, verbose=2)
feature_ls = ['YEAR', 'MONTH', 'DAY', 'OP_CARRIER', 'ORIGIN',
              'DEST', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'CRS_ELAPSED_TIME',
              'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'DIVERTED', 'CANCELLED' ]
le=LabelEncoder()

x=df[feature_ls]
y=df.FLIGHT_DELAY

#ut.missingPercent(x)
x=x.fillna(0, axis=1)
#ut.missingPercent(x)

x['OP_CARRIER']=le.fit_transform(x['OP_CARRIER'])
x['ORIGIN']=le.fit_transform(x['ORIGIN'])
x['DEST']=le.fit_transform(x['DEST'])

In [10]:
clf.fit(x,y)

building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   26.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=2,
            warm_start=False)

In [11]:
columns=['FEATURE', 'IMPORTANCE']
selected_Feature=pd.DataFrame(columns=columns)
for  idx in range(len(clf.feature_importances_)):
    selected_Feature.loc[idx]=[feature_ls[idx] ,clf.feature_importances_[idx]]
selected_Feature.sort_values('IMPORTANCE', ascending=False)

,FEATURE,IMPORTANCE
6,DEP_DELAY,0.362997
7,TAXI_OUT,0.135066
9,CRS_ELAPSED_TIME,0.122171
10,ACTUAL_ELAPSED_TIME,0.093196
11,AIR_TIME,0.057661
14,TAXI_IN,0.039270
12,DISTANCE,0.039141
13,WHEELS_ON,0.032182
8,WHEELS_OFF,0.030354
2,DAY,0.025966


In [12]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

df['DAY_OF_WEEK'] = df['FL_DATE'].dt.day_name()

In [13]:
variables_to_remove=['YEAR', 'MONTH', 'DAY']
df.drop(variables_to_remove, axis=1, inplace=True)

In [14]:
variables_to_remove=['ARR_DELAY','ACTUAL_ELAPSED_TIME' , 'AIR_TIME', 'CRS_DEP_TIME','CRS_ARR_TIME',
                     'FL_DATE','ARR_TIME' , 'WHEELS_ON', 'TAXI_IN','CANCELLED'] 
df.drop(variables_to_remove, axis=1, inplace=True)
print(df.shape)

(532339, 12)


In [15]:
delay_binary={'YES':0,'NO':1}

df['FLIGHT_DELAY']=df['FLIGHT_DELAY'].map(delay_binary)
df.head()

,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,DIVERTED,CRS_ELAPSED_TIME,DISTANCE,FLIGHT_DELAY,DAY_OF_WEEK
0,XE,DCA,EWR,1058.0,-2.0,18.0,1116.0,0.0,62.0,199.0,0,Thursday
1,XE,EWR,IAD,1509.0,-1.0,28.0,1537.0,0.0,82.0,213.0,1,Thursday
2,XE,EWR,DCA,1059.0,-1.0,20.0,1119.0,0.0,70.0,199.0,1,Thursday
3,XE,DCA,EWR,1249.0,9.0,10.0,1259.0,0.0,77.0,199.0,1,Thursday
4,XE,IAD,EWR,1705.0,-10.0,24.0,1729.0,0.0,105.0,213.0,1,Thursday


In [16]:
df['LAG_INFO']= df['ORIGIN']+ '--' +df['DEST']
df.shape, df.columns

((532339, 13),
 Index(['OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT',
        'WHEELS_OFF', 'DIVERTED', 'CRS_ELAPSED_TIME', 'DISTANCE',
        'FLIGHT_DELAY', 'DAY_OF_WEEK', 'LAG_INFO'],
       dtype='object'))

In [17]:
variables_to_remove=['DEST','ORIGIN','DEP_TIME']
df.drop(variables_to_remove, axis=1, inplace=True)
print(df.shape)

(532339, 10)


In [18]:
print(df['LAG_INFO'].nunique())
df.head()

4154


,OP_CARRIER,DEP_DELAY,TAXI_OUT,WHEELS_OFF,DIVERTED,CRS_ELAPSED_TIME,DISTANCE,FLIGHT_DELAY,DAY_OF_WEEK,LAG_INFO
0,XE,-2.0,18.0,1116.0,0.0,62.0,199.0,0,Thursday,DCA--EWR
1,XE,-1.0,28.0,1537.0,0.0,82.0,213.0,1,Thursday,EWR--IAD
2,XE,-1.0,20.0,1119.0,0.0,70.0,199.0,1,Thursday,EWR--DCA
3,XE,9.0,10.0,1259.0,0.0,77.0,199.0,1,Thursday,DCA--EWR
4,XE,-10.0,24.0,1729.0,0.0,105.0,213.0,1,Thursday,IAD--EWR


In [19]:
df['DEP_DELAY'] = df['DEP_DELAY'].fillna(0)
df['TAXI_OUT'] = df['TAXI_OUT'].fillna(0)
df['WHEELS_OFF'] = df['WHEELS_OFF'].fillna(0)

In [20]:
final_df=df.loc[:150000]

In [21]:
df=pd.get_dummies(final_df, drop_first=False)
df.columns

Index(['DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'DISTANCE', 'FLIGHT_DELAY', 'OP_CARRIER_9E', 'OP_CARRIER_AA',
       'OP_CARRIER_AS',
       ...
       'LAG_INFO_XNA--IAH', 'LAG_INFO_XNA--LAX', 'LAG_INFO_XNA--LGA',
       'LAG_INFO_XNA--MEM', 'LAG_INFO_XNA--MSP', 'LAG_INFO_XNA--ORD',
       'LAG_INFO_YAK--CDV', 'LAG_INFO_YAK--JNU', 'LAG_INFO_YUM--LAX',
       'LAG_INFO_YUM--PHX'],
      dtype='object', length=4162)

In [22]:
Y=df['FLIGHT_DELAY']
df.drop('FLIGHT_DELAY',  axis=1, inplace=True)

In [23]:
xdf=df

In [24]:
normal=MinMaxScaler()
normal.fit_transform(xdf)

array([[0.03278689, 0.07594937, 0.465     , ..., 0.        , 0.        ,
        0.        ],
       [0.03360656, 0.11814346, 0.64041667, ..., 0.        , 0.        ,
        0.        ],
       [0.03360656, 0.08438819, 0.46625   , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.0295082 , 0.08860759, 0.56458333, ..., 0.        , 0.        ,
        0.        ],
       [0.04016393, 0.06329114, 0.56166667, ..., 0.        , 0.        ,
        0.        ],
       [0.03032787, 0.02531646, 0.58875   , ..., 0.        , 0.        ,
        0.        ]])

In [25]:
clfdt=DecisionTreeClassifier()
lr=LogisticRegression()

In [26]:
x_train, x_test, y_train,y_test =train_test_split(xdf,Y, test_size=.20)

In [27]:
from sklearn.model_selection import KFold,cross_validate
n_splits = 5
kf = KFold(n_splits=n_splits,shuffle=True)
scoring=('r2','neg_mean_squared_error')

cv=cross_validate(lr,x_train,y_train,cv=kf,scoring=scoring,return_train_score=False)

In [29]:
cv_pre=cv.predict(x_test)

AttributeError: 'dict' object has no attribute 'predict'

In [29]:
lr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [30]:
y_pred_lr=lr.predict(x_test)
print("----------------------------- Model Evaluation----------------------- ")
print('Model Accuracy ', accuracy_score(y_pred_lr, y_test))
print("----------------------------- Model Evaluation----------------------- ")
print("Recall Score ",recall_score(y_pred_lr, y_test))
print("----------------------------- Model Evaluation----------------------- ")
print("Precssion Score ",precision_score(y_pred_lr, y_test))
print("----------------------------- Model Evaluation----------------------- ")
print("Roc AUC Score ",roc_auc_score(y_pred_lr, y_test))

----------------------------- Model Evaluation----------------------- 
Model Accuracy  0.8529049031698943
----------------------------- Model Evaluation----------------------- 
Recall Score  0.835393734123624
----------------------------- Model Evaluation----------------------- 
Precssion Score  0.9081369661266568
----------------------------- Model Evaluation----------------------- 
Roc AUC Score  0.8567738652711561


In [31]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test, y_pred_lr)

array([[10789,  2916],
       [ 1497, 14799]], dtype=int64)

In [34]:
clfdt=DecisionTreeClassifier()

In [35]:
clfdt.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [36]:
y_pred_dt=clfdt.predict(x_test)

In [39]:
confusion_matrix(y_test, y_pred_dt)

array([[10542,  3163],
       [ 2748, 13548]], dtype=int64)